# Proscenium Legal Demo

This notebook demonstrates

1. The use of an LLM to enrich of publically available legal opinions
2. Entity resolution
3. Query parsing
4. Context formation for question answering

## Prerequisites

1. Obtain a `TOGETHER_API_KEY` from https://together.ai/ and store it as an environment variable or Colab secret.
2. Obtain a Neo4j Sandbox https://neo4j.com/sandbox/



## Setup

In [ ]:
!git clone https://github.com/The-AI-Alliance/proscenium.git

In [ ]:
%cd proscenium

In [ ]:
!python -m pip install .

In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
from rich import print
from rich.panel import Panel
from rich.prompt import Prompt
import asyncio

import nest_asyncio
nest_asyncio.apply()

## Configure

The folowing cell defines

1. Get secrets for accessing Together and Neo4j
2. Hugging Face dataset id (NH caselaw)
3. Filenames (entity csv)
4. Model ids (for extraction, embedding, and generation)
5. Domain-specific logic for processing caselaw and user queries



In [ ]:
from google.colab import userdata

api_key = userdata.get('TOGETHER_API_KEY')
os.environ['TOGETHER_API_KEY'] = api_key

neo4j_uri = userdata.get('NEO4J_URI')
neo4j_username = userdata.get("NEO4J_USERNAME")
neo4j_password = userdata.get("NEO4J_PASSWORD")

from pathlib import Path
enrichment_jsonl_file = Path("enrichments.jsonl")

milvus_uri = "file:/grag-milvus.db"

import demo.domains.legal as domain


## Extract Knowledge Graph

In [ ]:
from proscenium.scripts.document_enricher import enrich_documents

docs_per_dataset = 4

enrich_documents(
    domain.retriever(docs_per_dataset),
    domain.doc_as_rich,
    enrichment_jsonl_file,
    domain.default_chunk_extraction_model_id,
    domain.chunk_extraction_template,
    domain.LegalOpinionChunkExtractions,
    domain.doc_enrichments,
    delay=2,
    verbose=True,
)


## Load Knowledge Graph

In [ ]:
from proscenium.verbs.know import knowledge_graph_client
from proscenium.scripts.knowledge_graph import load_knowledge_graph


In [ ]:
driver = knowledge_graph_client(neo4j_uri, neo4j_username, neo4j_password)

load_knowledge_graph(
    driver,
    enrichment_jsonl_file,
    domain.LegalOpinionEnrichments,
    domain.doc_enrichments_to_graph,
)

driver.close()


## Show Knowledge Graph

In [ ]:
driver = knowledge_graph_client(neo4j_uri, neo4j_username, neo4j_password)

domain.show_knowledge_graph(driver)

driver.close()

## Load Entity Resolver

In [ ]:
from proscenium.scripts.entity_resolver import load_entity_resolver

driver = knowledge_graph_client(neo4j_uri, neo4j_username, neo4j_password)

load_entity_resolver(
    driver,
    domain.resolvers,
    milvus_uri,
)

driver.close()

## Answer Question

In [ ]:
question = "How has Judge Kenison used Ballou v. Ballou to rule on cases?"

In [ ]:
from proscenium.scripts.graph_rag import answer_question
#from rich.panel import Panel

driver = knowledge_graph_client(neo4j_uri, neo4j_username, neo4j_password)

answer = answer_question(
    question,
    domain.default_query_extraction_model_id,
    milvus_uri,
    driver,
    domain.default_generation_model_id,
    domain.query_extract,
    domain.extract_to_context,
    domain.context_to_prompts,
    True,
)


if answer:
    print(Panel(answer, title="Answer"))
else:
    print("No answer")

driver.close()